In [1]:
import pandas as pd
import numpy as np 

In [2]:
#setting parameters 
np.random.seed(42)

# Parameters
num_instruments = 10
num_obs_insample = 200  # In-sample observations
num_obs_outsample = 100  # Out-of-sample observations

betas = np.random.uniform(0.5, 1.5, size=num_instruments)
factor_exposure = np.random.uniform(-1, 1, size=num_obs_insample + num_obs_outsample)


In [3]:
def generate_dataset(num_instruments,num_obs):
    # Generate alphas Z1 and Z2
    Z1 = np.random.uniform(-0.5, 0.5, size=(num_obs, num_instruments))
    Z2 = np.random.uniform(-0.5, 0.5, size=(num_obs, num_instruments))
    risk_factor = factor_exposure[:num_obs]
    errors = np.random.normal(0, 0.05, size=(num_obs, num_instruments))

    # Calculate returns
    returns = (
        risk_factor[:, np.newaxis] +
        Z1 / betas +
        errors
    )
    # Create in-sample and out-of-sample DataFrames
    columns = ['instrument', 'Z1', 'Z2', 'riskFactor', 'return']

    df = pd.DataFrame({
        'instrument': np.tile(np.arange(num_instruments), num_obs),
        'Z1': Z1.flatten(),
        'Z2': Z2.flatten(),
        'riskFactor': np.repeat(risk_factor, num_instruments),
        'return': returns.flatten()
    })
    return df 

In [4]:
in_sample_df = generate_dataset(num_instruments,num_obs_insample)
out_sample_df = generate_dataset(num_instruments,num_obs_outsample)

## Methodology 
Follow a two step linear regression approach  
1. **Regreess `returns` on `risk_factor` to estimate the risk factor sensitivity or $\beta$**
$$ R_i = \beta_f \cdot F + \epsilon$$
_vectorized_:
$$ \beta = (X ^\top X)^{-1}X^\top y$$
as: `beta = np.linalg.inv(X.T @ X) @ X.T @ y`
2. **Estimate alpha contribution**  
Regress residuals obtained from the first regression on the alpha columns: idio risk 
$$ \epsilon = \gamma_1 \cdot Z1 + \gamma_2 \cdot Z2 + \eta $$

The gamma returns are the coefficents for the alphas where position can be defined as 
$$ Position_i = \gamma_1 \cdot Z1_i + \gamma_2 \cdot Z2_i$$


In [5]:
# Define the design matrix (add a column of ones for the intercept)
X = in_sample_df['riskFactor'].values.reshape(-1, 1) # assume intercept at 1 
y = in_sample_df['return'].values

# Compute betas using the closed-form solution
beta = np.linalg.inv(X.T @ X) @ X.T @ y
#predict
y_pred = X @ beta 
#get the resids
residuals = y - y_pred

#Second regression / regress the idio 
# Each column can correspond to a predictor
X_2 = np.column_stack((in_sample_df['Z1'].values, in_sample_df['Z2'].values))
y_2 = residuals 
gammas = np.linalg.inv(X_2.T @ X_2) @ X_2.T @ y_2

positions = gammas[0] * in_sample_df['Z1'].values + gammas[1] * in_sample_df['Z2'].values
positions_normalized = positions / np.sum(np.abs(positions))


### Apply to outofsample dataset 

In [6]:
positions_outsample = gammas[0] * out_sample_df['Z1'].values + gammas[1] * out_sample_df['Z2'].values
positions_outsample_normalized = positions_outsample / np.sum(np.abs(positions_outsample))
portfolio_returns = positions_outsample_normalized * out_sample_df['return'].values

### Optimize on sharpe 

In [7]:
mean_return = np.mean(portfolio_returns)
std_return = np.std(portfolio_returns)
sharpe = mean_return / std_return 

In [9]:
def negative_sharpe(gammas):
    #reuse outofsample values
    positions_outsample = gammas[0] * out_sample_df['Z1'].values + gammas[1] * out_sample_df['Z2'].values 
    positions_outsample_normalized = positions_outsample / np.sum(np.abs(positions_outsample))
    portfolio_returns = positions_outsample_normalized * out_sample_df['return'].values
    mean_return = np.mean(portfolio_returns)
    std_return = np.std(portfolio_returns)
    return - mean_return / std_return 

In [11]:
from scipy.optimize import minimize 

initial_gammas = gammas

result = minimize(negative_sharpe, initial_gammas,method='Nelder-Mead')
optimized_gammas = result.x

#positions 

optimized_positions = optimized_gammas[0] * out_sample_df['Z1'].values + optimized_gammas[1] * out_sample_df['Z2'].values 
optimized_positions_normalized = optimized_positions / np.sum(np.abs(optimized_positions))
optimized_portfolio_returns = optimized_positions_normalized * out_sample_df['return'].values
optimized_mean_return = np.mean(optimized_portfolio_returns)
optimized_std_return = np.std(optimized_portfolio_returns)
optimized_sharpe = optimized_mean_return / optimized_std_return

### Feel correct since the alphas are randomized 

In [12]:
optimized_sharpe

0.45385491285115237